# Testing the model

## 1. Install dependencies

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade
     

!pip install langchain langchain_core einops accelerate transformers bitsandbytes scipy
     

!pip install xformers sentencepiece 
     

!pip install llama-index
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-langchain
%pip install llama-index-llms-huggingface


!pip install python-dotenv

Looking in indexes: https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 53.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import transformer classes for generation

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig

# Import torch for datatype attributes

import torch

import os
from dotenv import load_dotenv

In [ ]:
# Define model variable and load token from .env file

load_dotenv()

name = "meta-llama/Llama-2-70b-chat-hf"
token = env

## 2. Instantiate the tokenizer

In [4]:
# Instantiate tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    name, 
    cache_dir='./model/', 
    use_auth_token=token)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## 3. Instantiate the model

In [5]:
# First define quantization settings (to reduce memory usage and computational requirements), then instantiate model

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    name, 
    cache_dir='./model/', 
    use_auth_token=token, 
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    rope_scaling={"type": "dynamic", "factor": 2}) 


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/66.7k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

...

model-00013-of-00015.safetensors:   0%|          | 0.00/9.80G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/9.50G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/524M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## 4. Try the model

In [6]:
# Setup a prompt, pass it to the tokenizer and set up the text streamer

prompt = "### User:Who is the richest man in  \
          the world and what is his networth? \
          ### Assistant:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [7]:
# Run the model
output = model.generate(**inputs, 
                        streamer=streamer, 
                        use_cache=True, 
                        max_new_tokens=float('inf'))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


The richest man in the world is currently Bernard Arnault, with an estimated net worth of over $200 billion. He is the CEO of LVMH, a luxury goods conglomerate that owns brands such as Louis Vuitton, Moet Hennessy, and Hennessy. His net worth is estimated to be around $203 billion, according to Forbes.

However, it's important to note that the ranking of the richest people in the world can change constantly, as the net worth of individuals can fluctuate based on various factors such as market conditions, business performance, and investments.

It's also worth mentioning that there are other billionaires who are not far behind Bernard Arnault in terms of net worth, such as Jeff Bezos, Bill Gates, and Warren Buffett. These individuals have also built impressive fortunes through their successful business ventures and investments.


In [10]:
# Convert the output tokens back to text 

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
output_text

"### User:Who is the richest man in            the world and what is his networth?           ### Assistant: The richest man in the world is currently Bernard Arnault, with an estimated net worth of over $200 billion. He is the CEO of LVMH, a luxury goods conglomerate that owns brands such as Louis Vuitton, Moet Hennessy, and Hennessy. His net worth is estimated to be around $203 billion, according to Forbes.\n\nHowever, it's important to note that the ranking of the richest people in the world can change constantly, as the net worth of individuals can fluctuate based on various factors such as market conditions, business performance, and investments.\n\nIt's also worth mentioning that there are other billionaires who are not far behind Bernard Arnault in terms of net worth, such as Jeff Bezos, Bill Gates, and Warren Buffett. These individuals have also built impressive fortunes through their successful business ventures and investments."

# RAG with LlamaIndex

## 1. Import the prompt wrapper for llamaindex

In [11]:

# Import the prompt wrapper...but for llama index

from llama_index.core.prompts.prompts import SimpleInputPrompt



# Create a system prompt 
system_prompt = """[INST] <>
You are a helpful, respectful and honest assistant. Always answer as 
helpfully as possible, while being safe. Your answers should not include
any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain 
why instead of answering something not correct. If you don't know the answer 
to a question, please don't share false information.

Your goal is to provide answers relating to the financial performance of 
the company.<>
"""
# Throw together the query wrapper
query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")


# Complete the query prompt
query_wrapper_prompt.format(query_str='hello')

'hello [/INST]'

## 2. Create a HF LLM using the llama index HuggingFaceLLM wrapper

In [12]:
# Import the llama index HF Wrapper
from llama_index.llms.huggingface import HuggingFaceLLM

# # Create a HF LLM using the llama index wrapper 
llm = HuggingFaceLLM(context_window=4096,
                    max_new_tokens=256,
                    system_prompt=system_prompt,
                    query_wrapper_prompt=query_wrapper_prompt,
                    model=model,
                    tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
The model `StabilityAI/stablelm-tuned-alpha-3b` and tokenizer `meta-llama/Llama-2-70b-chat-hf` are different, please ensure that they are compatible.


## 3. Import embedding classes and define embedding instance

In [13]:
# Import embeddings wrapper
from llama_index.embeddings.langchain import LangchainEmbedding

# Import HF embeddings - need these to represent document chunks
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
     

# # Create and dl embeddings instance  
embeddings=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 4. Define a new service context instance with our transformers model as the llm. 
### (This is needed because the default llm for LlamaIndex is OpenAI, not Llama).

In [14]:

# Import dependencies to change service context. This is needed because by default, LlamaIndex works with OpenAI

from llama_index.core import set_global_service_context
from llama_index.core import ServiceContext
     

# # Create new service context instance
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embeddings
)

# # Set the service context
set_global_service_context(service_context)

/tmp/ipykernel_147/4286978408.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


## 5. Import dependencies for loading, storing and then retrieving documents.
### (download_loader to load documents and VectorStoreIndex for storing the document chunks)

In [17]:
!pip install --upgrade pymupdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 20.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 66.0 MB/s eta 0:00:00a 0:00:01


In [18]:

# Import dependencies to load documents 

from llama_index.core import VectorStoreIndex, download_loader
from pathlib import Path
     

# Download PDF Loader 
PyMuPDFReader = download_loader("PyMuPDFReader")
# Create PDF Loader
loader = PyMuPDFReader()
# Load documents 
documents = loader.load(file_path=Path('annual_report.pdf'), metadata=True)
     

# Create an index - we'll be able to query this in a sec
index = VectorStoreIndex.from_documents(documents)

/tmp/ipykernel_147/1009674855.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PyMuPDFReader = download_loader("PyMuPDFReader")


## 6. Set up a query_engine instance that allows us to query the Vector Database using our llm (Llama 2).

In [41]:
# Setup index query engine using LLM 
query_engine = index.as_query_engine()
     

# Test out a query in natural
response = query_engine.query("What was Revenue in FY21 and FY22?")
response.response

' Revenue in FY21 was £145.8 million, and in FY22 it was £163.8 million, which is an increase of 12%.'

### Yay! The result is 100% accurate!!